In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.draw import disk, circle_perimeter, rectangle_perimeter
from scipy.ndimage import gaussian_filter
import csv
import torch
import torchvision
from torchvision import transforms as T
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from PIL import Image

#LOAD EXISTING MODEL
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=False)
anchor_generator = AnchorGenerator(sizes=((16,), (32,), (64,), (128,), (256,)), aspect_ratios=((0.5, 1.0, 2.0),) * 5)
model.rpn.anchor_generator = anchor_generator
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 5)
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, 5)
model.rpn.post_nms_top_n_train = 2000
model.rpn.post_nms_top_n_test = 2500
model.roi_heads.detections_per_img = 2000

model_path = 'MODELPATH' #path to the model
saved_state_dict = torch.load(model_path)
model.load_state_dict(saved_state_dict)
model = model.to(device)

In [ ]:
#IMAGE EVAL
torch.cuda.empty_cache()

img_path = "example/BF/0.png"
img = Image.open(img_path).convert('RGB')
full_img_array = np.array(img)
transform = T.ToTensor()

# Define color mapping for labels
label_color_map = {
    1: (192, 0, 0),  # Red
    2: (0, 0, 0),  # Green
    3: (255, 255, 0),  #yellow
    4: (0, 0, 255),  # blue
}

def add_box_outlines(image, boxes, labels):
    for box, label in zip(boxes, labels):
        color = label_color_map.get(label.item(), (255, 255, 255))  # Default to white if label not found
        box = box.cpu().numpy().astype(np.int32)
        rr, cc = rectangle_perimeter((box[1], box[0]), end=(box[3], box[2]), shape=image.shape)
        image[rr, cc] = color
    return image

def add_masks(image, masks, labels, alpha=0.5):
    for mask, label in zip(masks, labels):
        color = label_color_map.get(label.item(), (255, 255, 255))
        mask_np = mask.cpu().numpy().squeeze()
        for i in range(3):
            image[:, :, i] = np.where(mask_np > 0.5, image[:, :, i] * (1 - alpha) + color[i] * alpha, image[:, :, i])
    return image

img_tensor = transform(img).to(device)

model.eval()
with torch.no_grad():
    pred = model([img_tensor])

confidence_threshold = 0.3
high_conf_indices = [i for i, score in enumerate(pred[0]['scores']) if score > confidence_threshold]
high_conf_masks = pred[0]['masks'][high_conf_indices]
high_conf_boxes = pred[0]['boxes'][high_conf_indices]
high_conf_labels = pred[0]['labels'][high_conf_indices]

# Apply masks and boxes to the image
img_with_masks = add_masks(np.copy(full_img_array), high_conf_masks, high_conf_labels)
img_with_boxes_and_masks = add_box_outlines(img_with_masks, high_conf_boxes, high_conf_labels)
combined_image = np.concatenate([full_img_array, img_with_boxes_and_masks], axis=1)

# Display the final image
plt.figure(figsize=(20, 10))
plt.imshow(combined_image)
plt.axis('off')
plt.show()

In [ ]:
#Calculate the radii of the GUVs
guv_radii = []

for box, label in zip(high_conf_boxes_list, high_conf_labels_list):
    if label == 1:  # Only consider GUVs
        # Calculate the width and height of the bounding box
        width = box[2] - box[0]
        height = box[3] - box[1]
        
        # Calculate the average radius (assuming the GUV is approximately circular)
        radius = 0.5 * (width + height) / 2
        guv_radii.append(radius.cpu().numpy())

# Plotting the distribution of GUV radii
plt.figure(figsize=(10, 6))
plt.hist(guv_radii, bins=60, color='red')
plt.title('GUV Radii Distribution')
plt.xlabel('Radius (pixels)')
plt.ylabel('Frequency')
plt.grid(True)
plt.xlim(0,45)
plt.ylim(0,70)
plt.show()